In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from tpot import TPOTClassifier
import xgboost as xgboost
from xgboost import XGBClassifier

%matplotlib inline
sns.set() #adoptando el estilo de seaborn 

ModuleNotFoundError: No module named 'tpot'

In [ ]:
df_pago = pd.read_table("../data/raw/TB_PAGOS_CL.tsv", encoding="latin-1")
df_pago.info()

In [4]:
df_pago[df_pago.POLICY_NUMBER == 1698663309999]

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS
3973351,1698663309999,2017-08,2017-07,3888,TM,CT


In [22]:
#creo variable para ordenar por fecha
df_pago["DUE_DATE_mes"]  = df_pago.DUE_DATE.apply(lambda x : int(x.split("-")[1]))
df_pago["DUE_DATE_anio"] = df_pago.DUE_DATE.apply(lambda x : int(x.split("-")[0]))
df_pago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4111565 entries, 0 to 4111564
Data columns (total 8 columns):
POLICY_NUMBER      int64
DUE_DATE           object
PAYMENT_DATE       object
PAYMENT_AMOUNT     object
MODE_OF_PAYMENT    object
PAYMENT_STATUS     object
DUE_DATE_mes       int64
DUE_DATE_anio      int64
dtypes: int64(3), object(5)
memory usage: 251.0+ MB


In [27]:
df_pago.sort_values(["DUE_DATE_anio", "DUE_DATE_mes"], inplace=True)
df_pago.head(20)

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio
0,77320269999,2004-01,2004-01,420,PP,CT,1,2004
7,79741709999,2004-01,2004-01,1088,PP,CT,1,2004
15,86520099999,2004-01,2004-01,660,PP,CT,1,2004
25,92068329999,2004-01,2004-01,1776,PP,CT,1,2004
37,100004239999,2004-01,2004-01,405,PP,CT,1,2004
42,108013639999,2004-01,2004-01,525,PP,CT,1,2004
49,119100429999,2004-01,2004-01,1098,PP,CT,1,2004
58,127372969999,2004-01,2004-01,1356,PP,CT,1,2004
70,130623839999,2004-01,NaN,80,PP,EP,1,2004
72,134881649999,2004-01,2004-01,1695,PP,CT,1,2004


### cargo bd baja

In [24]:
df_baja = pd.read_table("../data/raw/TB_BAJAS_CL.tsv", encoding="latin-1")
df_baja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99025 entries, 0 to 99024
Data columns (total 3 columns):
POLICY_NUMBER          99025 non-null int64
CANCELLATION_DATE      99025 non-null object
CANCELLATION_REASON    16917 non-null object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


# Busco elementos para control de la lògica

In [42]:
#muestro las primeras 3 polizas que aparecen en bajas
df_pago[df_pago["POLICY_NUMBER"].isin(df_pago["POLICY_NUMBER"].unique()[684333:684334])].sort_values(by="POLICY_NUMBER")

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio
688813,670578219999,2009-01,2009-02,250,TM,CT,1,2009


In [43]:
df_baja[df_baja["POLICY_NUMBER"] == 670578219999]

,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON


#### Conclusión baja y pago
A partir del análisis se observa que el mes de pago corresponde al siguiente mes, por ahora apareciendo siempre el último mes anterior al mes de pedido de baja. Es decir, siempre que se pidio de baja aparece el su mes de pago correspondiente

## Creo comparación de fechas

In [59]:
#obtengo aquellas polizas que tengan más de 1 registro (pago)
pagosN = pd.DataFrame(df_pago.groupby("POLICY_NUMBER").POLICY_NUMBER.count()>1)
pagosN.columns = ["logic"]
pagosN.reset_index(level=0, inplace=True)
#y luego se puede obtener los ids de las columnas de interés y filtrar en una lista
ids_pol = pagosN[pagosN["logic"] == True]["POLICY_NUMBER"].unique()

In [64]:
df_pago.head()

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio
0,77320269999,2004-01,2004-01,420,PP,CT,1,2004
7,79741709999,2004-01,2004-01,1088,PP,CT,1,2004
15,86520099999,2004-01,2004-01,660,PP,CT,1,2004
25,92068329999,2004-01,2004-01,1776,PP,CT,1,2004
37,100004239999,2004-01,2004-01,405,PP,CT,1,2004


In [125]:
from datetime import datetime, timedelta

In [69]:
#analicemos si hay discontinuos entre polizas en los meses de pago

#generamos una fecha fija
df_pago["fecha"] = df_pago.apply(lambda x : datetime.strptime("{}-{}-01".format(x["DUE_DATE_anio"], x["DUE_DATE_mes"]), "%Y-%m-%d"), axis=1)
df_pago.head()

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha
0,77320269999,2004-01,2004-01,420,PP,CT,1,2004,2004-01-01
7,79741709999,2004-01,2004-01,1088,PP,CT,1,2004,2004-01-01
15,86520099999,2004-01,2004-01,660,PP,CT,1,2004,2004-01-01
25,92068329999,2004-01,2004-01,1776,PP,CT,1,2004,2004-01-01
37,100004239999,2004-01,2004-01,405,PP,CT,1,2004,2004-01-01


In [104]:
df_pago.drop("next_f",axis=1)
df_pago["next_f"] = df_pago.groupby("POLICY_NUMBER").fecha.shift(1)
df_pago.head(40)
#for i in ids_pol:
#    df_pago.loc[df_pago.POLICY_NUMBER == i, ["next_f"]] = df_pago.loc[df_pago.POLICY_NUMBER == i]["fecha"].shift(1)

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha,next_f
0,77320269999,2004-01,2004-01,420,PP,CT,1,2004,2004-01-01,NaT
7,79741709999,2004-01,2004-01,1088,PP,CT,1,2004,2004-01-01,NaT
15,86520099999,2004-01,2004-01,660,PP,CT,1,2004,2004-01-01,NaT
25,92068329999,2004-01,2004-01,1776,PP,CT,1,2004,2004-01-01,NaT
37,100004239999,2004-01,2004-01,405,PP,CT,1,2004,2004-01-01,NaT
42,108013639999,2004-01,2004-01,525,PP,CT,1,2004,2004-01-01,NaT
49,119100429999,2004-01,2004-01,1098,PP,CT,1,2004,2004-01-01,NaT
58,127372969999,2004-01,2004-01,1356,PP,CT,1,2004,2004-01-01,NaT
70,130623839999,2004-01,NaN,80,PP,EP,1,2004,2004-01-01,NaT
72,134881649999,2004-01,2004-01,1695,PP,CT,1,2004,2004-01-01,NaT


In [107]:
df_pago.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4111565 entries, 0 to 4111564
Data columns (total 10 columns):
POLICY_NUMBER      int64
DUE_DATE           object
PAYMENT_DATE       object
PAYMENT_AMOUNT     object
MODE_OF_PAYMENT    object
PAYMENT_STATUS     object
DUE_DATE_mes       int64
DUE_DATE_anio      int64
fecha              datetime64[ns]
next_f             datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(5)
memory usage: 505.1+ MB


In [108]:
df_pago["next_f"] = df_pago.next_f.apply(lambda x : x if pd.isnull(x) else datetime(x.year + (x.month // 12), ((x.month % 12) + 1), 1) )
df_pago.head()

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha,next_f
0,77320269999,2004-01,2004-01,420,PP,CT,1,2004,2004-01-01,NaT
7,79741709999,2004-01,2004-01,1088,PP,CT,1,2004,2004-01-01,NaT
15,86520099999,2004-01,2004-01,660,PP,CT,1,2004,2004-01-01,NaT
25,92068329999,2004-01,2004-01,1776,PP,CT,1,2004,2004-01-01,NaT
37,100004239999,2004-01,2004-01,405,PP,CT,1,2004,2004-01-01,NaT


In [109]:
df_pago.loc[df_pago.POLICY_NUMBER == 77320269999, ["fecha", "next_f"]]

,fecha,next_f
0,2004-01-01,NaT
1,2004-02-01,2004-02-01
2,2004-03-01,2004-03-01
3,2004-04-01,2004-04-01
4,2004-05-01,2004-05-01
5,2004-06-01,2004-06-01
6,2004-07-01,2004-07-01


In [139]:
df_pago[df_pago["PAYMENT_STATUS"] == "EP"]

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha,next_f,siguiente
70,130623839999,2004-01,NaN,80,PP,EP,1,2004,2004-01-01,NaT,NaT
521,259238219999,2004-01,NaN,34,TM,EP,1,2004,2004-01-01,NaT,NaT
533,259261449999,2004-01,NaN,39,CC,EP,1,2004,2004-01-01,NaT,NaT
676,260077109999,2004-01,NaN,56,CC,EP,1,2004,2004-01-01,NaT,NaT
721,260397989999,2004-01,NaN,54,CC,EP,1,2004,2004-01-01,NaT,NaT
761,260796429999,2004-01,NaN,44,CC,EP,1,2004,2004-01-01,NaT,NaT
798,261096849999,2004-01,NaN,55,CC,EP,1,2004,2004-01-01,NaT,NaT
882,261656109999,2004-01,NaN,44,CC,EP,1,2004,2004-01-01,NaT,NaT
1051,262326259999,2004-01,NaN,43,CC,EP,1,2004,2004-01-01,NaT,NaT
1088,262516119999,2004-01,NaN,39,CC,EP,1,2004,2004-01-01,NaT,NaT


In [133]:
#comparo las fechas
df_pago["siguiente"] = (df_pago.fecha - df_pago.next_f) //30
df_pago["siguiente"].value_counts()

0 days 00:00:00      151500
-2 days +23:12:00      2985
1 days 00:48:00        2805
1 days 00:00:00        1769
-1 days +00:00:00      1586
2 days 00:48:00         947
3 days 01:36:00         502
0 days 22:24:00         391
-1 days +01:36:00       373
4 days 01:36:00         320
5 days 02:24:00         295
3 days 00:48:00         269
0 days 23:12:00         251
2 days 01:36:00         213
1 days 23:12:00         192
3 days 00:00:00         153
2 days 00:00:00         150
12 days 04:00:00        147
5 days 00:48:00         146
10 days 03:12:00        139
13 days 04:48:00        138
4 days 00:00:00         137
11 days 04:00:00        133
4 days 02:24:00         132
7 days 01:36:00         128
14 days 05:36:00        125
12 days 04:48:00        125
11 days 03:12:00        112
8 days 02:24:00         110
9 days 03:12:00         105
                      ...  
26 days 08:00:00          1
65 days 23:12:00          1
102 days 10:24:00         1
15 days 03:12:00          1
34 days 12:00:00    

### Se identifica desfasaje en algunos de los casos 

In [134]:
df_pago[df_pago["siguiente"] < timedelta(1, 0)]

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha,next_f,siguiente
1520,269915309999,2004-01,2004-02,-104,CC,CT,1,2004,2004-01-01,2004-02-01,-2 days +23:12:00
1,77320269999,2004-02,2004-02,420,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
8,79741709999,2004-02,2004-02,1088,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
16,86520099999,2004-02,2004-02,660,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
26,92068329999,2004-02,2004-02,1776,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
38,100004239999,2004-02,2004-02,405,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
43,108013639999,2004-02,2004-02,525,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
50,119100429999,2004-02,2004-02,1098,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
59,127372969999,2004-02,2004-02,1356,PP,CT,2,2004,2004-02-01,2004-02-01,0 days 00:00:00
71,130623839999,2004-02,NaN,80,PP,EP,2,2004,2004-02-01,2004-02-01,0 days 00:00:00


In [138]:

df_pago.loc[df_pago.POLICY_NUMBER == 269915309999]

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,DUE_DATE_mes,DUE_DATE_anio,fecha,next_f,siguiente
1519,269915309999,2004-01,2004-02,104,BA,CT,1,2004,2004-01-01,NaT,NaT
1520,269915309999,2004-01,2004-02,-104,CC,CT,1,2004,2004-01-01,2004-02-01,-2 days +23:12:00


In [136]:

df_baja.loc[df_baja.POLICY_NUMBER == 79741709999]

,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON
2525,79741709999,30/09/2004,NaN
